In [2]:
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
#import scipy.spatial.distance as dist
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
def dtp(dist_mat):
    N, M = dist_mat.shape
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1,N+1):
        cost_mat[i, 0]  = np.inf
    for j in range(1, M+1):
        cost_mat[0, j] = np.inf
    traceback_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            min_list = [cost_mat[i, j], # match = 0
                        cost_mat[i, j+1],   #insert = 1
                        cost_mat[i+1, j]]   # deletion = 2
            index_min = np.argmin(min_list)
            cost_mat[i+1,j+1] = dist_mat[i, j] + min_list[index_min]
            traceback_mat[i,j] = index_min 
    i = N-1
    j = M -1
    path = [(i,j)]
    while i > 0 or j > 0:
        tb_type = traceback_mat[i,j]
        if tb_type == 0: # đi chéo
            i = i-1
            j = j-1
        elif tb_type == 1: # đi xuống
            i = i - 1
        elif tb_type == 2: # đi ngang
            j = j - 1
        path.append((i,j))
    cost_mat = cost_mat[1:, 1:]
    return path[::-1], cost_mat,traceback_mat


In [4]:
def distance_matrix(mat1, mat2):
    N = mat1.shape[0]
    M = mat2.shape[0]
    dist_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = np.linalg.norm(mat1[i, :] - mat2[j,:] )
    return dist_mat
    

In [5]:
def calculate_angle(a,b,c):
    a, b, c = np.array(a), np.array(b),  np.array(c)

    radians = np.arctan2(c[1]- b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    if angle > 180:
        angle = 360 - 180
    return angle

In [10]:
def extract_vid(vid_path):
    cap = cv2.VideoCapture(vid_path)
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    matrix = []
    count = 0
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        while(cap.isOpened()):
            try:
                ret, frame = cap.read()
                count += 1
            #Change from BGR to RGB
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
                img.flags.writeable = False

                #Make detection
                result = pose.process(img)

                # Back the original color channels
                img.flags.writeable = True
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                #Extract landmarks 
                try:
                    landmarks = result.pose_landmarks.landmark
                    #print(landmarks)
                except:
                    pass
                # Landmarks: là 1 list chứa tọa độ 33 điểm trên cơ thể
                theta = []
                for i in range(33):
                    for j in range(i+1, 33):
                        for k in range(j+1,33): 
                            a = [landmarks[i].x, landmarks[i].y]
                            b = [landmarks[j].x, landmarks[j].y]
                            c = [landmarks[k].x, landmarks[k].y]
                            angle = calculate_angle(a, b, c)
                            theta.append(angle)
                matrix.append(theta)

            except:
                pass
            if count == 300:
                break
        
        return np.array(matrix), N



In [9]:
mat1,n = extract_vid("Sources/viddtw1.mp4")
mat1.shape

(200, 5456)

In [11]:
mat2, m= extract_vid("Sources/viddtw2.mp4")
mat2.shape

(300, 5456)

In [12]:
dist_mat = distance_matrix(mat1, mat2)
dist_mat

array([[3730.84263289, 3696.89236445, 3764.37874079, ..., 3276.28175709,
        3280.05186328, 3278.52130387],
       [3758.68204255, 3721.1284795 , 3786.36987434, ..., 3287.92006129,
        3289.61833345, 3286.89990448],
       [3876.00319539, 3841.10443918, 3904.50177387, ..., 3084.42644337,
        3088.75336725, 3108.6453415 ],
       ...,
       [3785.1521674 , 3741.47355262, 3774.35738571, ..., 3486.0973287 ,
        3501.63871419, 3530.70489407],
       [3871.42604766, 3830.00056284, 3868.19778997, ..., 3556.87608192,
        3572.49321789, 3603.2331145 ],
       [3857.39999583, 3813.32468099, 3854.76665943, ..., 3424.09288271,
        3439.42636488, 3465.14516504]])

In [13]:
dist_mat.shape

(200, 300)

In [14]:
dist_mat[0,1]

3696.8923644466895

In [15]:
path, cost_mat, tbm= dtp(dist_mat)
# print(cost_mat[n - 1, m - 1])
# print(cost_mat[n - 1, m - 1]/(n + m))

# plt.figure(figsize=(6, 4))
# plt.subplot(121)
# plt.title("Distance matrix")
# plt.imshow(dist_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
# plt.subplot(122)
# plt.title("Cost matrix")
# plt.imshow(cost_mat, cmap=plt.cm.binary, interpolation="nearest", origin="lower")
# x_path, y_path = zip(*path)
# plt.plot(y_path, x_path);

In [16]:
path[-100:]

[(120, 200),
 (121, 201),
 (122, 202),
 (123, 203),
 (124, 204),
 (125, 205),
 (126, 206),
 (127, 207),
 (128, 208),
 (129, 209),
 (130, 210),
 (131, 211),
 (132, 212),
 (133, 213),
 (134, 214),
 (135, 215),
 (136, 216),
 (137, 217),
 (138, 218),
 (139, 219),
 (140, 220),
 (141, 221),
 (142, 222),
 (143, 223),
 (144, 224),
 (145, 225),
 (146, 226),
 (147, 227),
 (148, 228),
 (149, 229),
 (150, 230),
 (151, 231),
 (152, 232),
 (153, 233),
 (154, 234),
 (155, 235),
 (156, 236),
 (157, 237),
 (158, 238),
 (159, 239),
 (160, 240),
 (161, 241),
 (162, 242),
 (163, 243),
 (164, 244),
 (165, 245),
 (166, 246),
 (167, 247),
 (168, 248),
 (169, 249),
 (170, 250),
 (171, 251),
 (172, 252),
 (172, 253),
 (172, 254),
 (173, 255),
 (173, 256),
 (173, 257),
 (173, 258),
 (173, 259),
 (173, 260),
 (173, 261),
 (173, 262),
 (173, 263),
 (173, 264),
 (173, 265),
 (173, 266),
 (173, 267),
 (173, 268),
 (173, 269),
 (173, 270),
 (173, 271),
 (173, 272),
 (173, 273),
 (174, 274),
 (175, 275),
 (176, 276),

In [14]:
cap1 = cv2.VideoCapture("Sources/viddtw1.mp4")
cap2 = cv2.VideoCapture('Sources/viddtw2.mp4')

while cap1.isOpened() or cap2.isOpened():
    for i in path:
        cap1.set(1,i[0])
        cap2.set(1, i[1])  # Where frame_no is the frame you want
        okay1  , frame1 = cap1.read()
        okay2 , frame2 = cap2.read()

        if okay1:
            #hsv1 = cv2.cvtColor(frame1 , cv2.COLOR_BGR2HSV)
            cv2.imshow('trainer' , frame1)

        if okay2:
            #hsv2 = cv2.cvtColor(frame2 , cv2.COLOR_BGR2HSV)
            cv2.imshow('user' , frame2)

        if not okay1 or not okay2:
            print('Cant read the video , Exit!')
            break

        if cv2.waitKey(20) & 0xFF == ord('q'):
            break
    
    if cv2.waitKey(20) & 0xFF == ord('q'):
        break 


cap1.release()
cap2.release()
cv2.destroyAllWindows()